## Bad Performance on DSC

In [1]:
import os
import pandas as pd

# Path to the root folder (adjust as needed)
models_root = 'final_14dec2024_results/jhh_results - table 2 in paper'

bad_performance_threshold = 0.3

# Dictionary to store results
low_performance_dict = {}

# Loop through each model folder
for model_name in os.listdir(models_root):
    model_path = os.path.join(models_root, model_name)
    dsc_path = os.path.join(model_path, 'dsc.csv')

    if os.path.isfile(dsc_path):
        dsc_df = pd.read_csv(dsc_path)

        # Initialize dictionary for this model
        model_results = {}

        for organ in dsc_df.columns[1:]:  # Skip 'name' column
            if organ in dsc_df:
                low_perf_ids = dsc_df[dsc_df[organ] < bad_performance_threshold]['name'].tolist()
                model_results[organ] = low_perf_ids

        low_performance_dict[model_name] = model_results

#### Certain models are terrible at certain organs

In [2]:
# Now count how many low-performance cases there are per model per organ
counts_dict = {
    model: {organ: len(ct_ids) for organ, ct_ids in organs.items()}
    for model, organs in low_performance_dict.items()
}

# Convert to DataFrame for visualization
counts_df = pd.DataFrame.from_dict(counts_dict, orient='index').fillna(0).astype(int)

# Display or save the DataFrame
counts_df
counts_df.style.background_gradient(cmap='Blues')

,aorta,gall_bladder,kidney_left,kidney_right,liver,pancreas,postcava,spleen,stomach
UNETR,139,235,44,40,11,147,76,37,16
U-Net_CLIP,32,60,25,18,7,43,12,14,7
Diff-UNet,15,98,30,28,12,65,15,26,12
LHU-Net,14,89,30,26,10,64,17,20,10
NexToU,17,146,38,35,15,93,16,33,18
Swin_UNETR_CLIP,25,95,26,24,9,46,19,17,13
STU-Net H,12,51,27,25,10,60,11,17,9
nnU-Net_ResEncL,13,65,26,25,11,63,12,22,10
SegVol,20,188,27,26,11,84,18,24,12
UNEST,137,278,35,37,11,85,51,21,16


In [3]:
low_performance_dict.keys()

dict_keys(['UNETR', 'U-Net_CLIP', 'Diff-UNet', 'LHU-Net', 'NexToU', 'Swin_UNETR_CLIP', 'STU-Net H', 'nnU-Net_ResEncL', 'SegVol', 'UNEST', 'MedNeXt', 'nnU-Net U-Net', 'MedFormer', 'Swin_UNETR', 'STU-Net B', 'STU-Net L', 'UCTransNet', 'UniSeg', 'SAM-Adapter'])

In [4]:
len(list(low_performance_dict.keys()))

19

In [5]:
import pandas as pd

# Extract a dictionary of sets for LIVER only:
#   model_bad_cts[model_name] = {all CTs flagged by this model for liver}
model_bad_cts = {}
for model_name, organs_dict in low_performance_dict.items():
    # If "liver" is one of the organs in the dictionary:
    if "liver" in organs_dict:
        # Convert the list of "bad" CTs to a set for easier union/counting
        model_bad_cts[model_name] = set(organs_dict["liver"])
    else:
        # Or you can store an empty set if "liver" is not in the dictionary
        model_bad_cts[model_name] = set()

# 1) Create a union of all CTs flagged by at least one model
all_bad_cts = set()
for ct_set in model_bad_cts.values():
    all_bad_cts |= ct_set  # union

# 2) Count how many models flagged each CT
ct_flag_count = {}
for ct_id in all_bad_cts:
    count = sum(ct_id in ct_set for ct_set in model_bad_cts.values())
    ct_flag_count[ct_id] = count

# 3) Put it into a DataFrame and sort by number of flags descending
liver_union_df = pd.DataFrame.from_dict(
    ct_flag_count, orient='index', columns=['num_models_flagged']
).sort_values(by='num_models_flagged', ascending=False)

# 4) (Optional) Style the DataFrame with a background gradient
liver_union_df_styled = liver_union_df.style.background_gradient(cmap='Blues')

# Finally, display or save the styled DataFrame
liver_union_df_styled

,num_models_flagged
BDMAP_A0000761,19
BDMAP_A0000637,19
BDMAP_A0002501,19
BDMAP_A0000778,19
BDMAP_V0001722,18
BDMAP_V0000726,17
BDMAP_A0000763,17
BDMAP_V0001931,17
BDMAP_V0001954,17
BDMAP_V0001936,16
